# Fine Tuning

## Loading the libraries

In [24]:
from openai import OpenAI
import os
from dotenv import load_dotenv
import json
import numpy as np
from collections import defaultdict

## Setting the Environment

In [25]:
load_dotenv()

OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

## Validating the fine-tuning data file

In [26]:
data_path = "headphone-advice.jsonl"

# Load the dataset
with open(data_path, 'r', encoding='utf-8') as f:
    dataset = [json.loads(line) for line in f]

# Initial dataset stats
print("Num examples:", len(dataset))
print("First example:")
for message in dataset[0]["messages"]:
    print(message)



# Format error checks
format_errors = defaultdict(int)

for ex in dataset:
    if not isinstance(ex, dict):
        format_errors["data_type"] += 1
        continue
        
    messages = ex.get("messages", None)
    if not messages:
        format_errors["missing_messages_list"] += 1
        continue
        
    for message in messages:
        if "role" not in message or "content" not in message:
            format_errors["message_missing_key"] += 1
        
        if any(k not in ("role", "content", "name", "function_call") for k in message):
            format_errors["message_unrecognized_key"] += 1
        
        if message.get("role", None) not in ("system", "user", "assistant", "function"):
            format_errors["unrecognized_role"] += 1
            
        content = message.get("content", None)
        function_call = message.get("function_call", None)
        
        if (not content and not function_call) or not isinstance(content, str):
            format_errors["missing_content"] += 1
    
    if not any(message.get("role", None) == "assistant" for message in messages):
        format_errors["example_missing_assistant_message"] += 1

if format_errors:
    print("Found errors:")
    for k, v in format_errors.items():
        print(f"{k}: {v}")
else:
    print("No errors found")

Num examples: 10
First example:
{'role': 'system', 'content': 'Headphone buying advice chatbot'}
{'role': 'user', 'content': 'What are the key features to consider when buying headphones?'}
{'role': 'assistant', 'content': 'When buying headphones, consider factors like sound quality, comfort, wireless capability, battery life, and brand reputation.'}
No errors found


## Creating the client and uploading the fine tuning file

In [27]:
client = OpenAI()

client.files.create(
  file=open("headphone-advice.jsonl", "rb"),
  purpose="fine-tune"
)

FileObject(id='file-JPqxy0Z0h0kuFLVDUjEBogSA', bytes=3769, created_at=1701704717, filename='headphone-advice.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None)

## Creating the fine tuning job

In [29]:
client.fine_tuning.jobs.create(
  training_file="headphone-advice.jsonl",
  model="gpt-3.5-turbo"
)

BadRequestError: Error code: 400 - {'error': {'message': 'invalid training_file: headphone-advice.jsonl', 'type': 'invalid_request_error', 'param': 'training_file', 'code': None}}

## Showing the status of the fine tuning job

In [ ]:
client.fine_tuning.jobs.list(limit=10)